# Coursera Capstone for IBM Data Science

### Segmenting and Clustering Neighborhoods in Toronto



In [1]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
import requests

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [3]:
class Scrapy:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(self.parse_html_table(table))\
                    for table in soup.find_all('table', class_="wikitable sortable")]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
            for row in table.find_all('tr'):
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        n_columns = len(td_tags)
                        
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
hp = Scrapy()
table = hp.parse_url(url)[0] 
table.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


#### Remove "not assigned"

In [5]:
table = table[table.Borough != 'Not assigned']
table.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
10,M9A,Etobicoke,Islington Avenue\n
11,M1B,Scarborough,Rouge\n
12,M1B,Scarborough,Malvern\n


### Clean up the data

In [6]:
print(list(table))
table = table.replace('\n',' ', regex=True)
table.head(10)

['Postcode', 'Borough', 'Neighbourhood\n']


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [7]:
table = table[table['Neighbourhood\n'] != 'Not assigned']
table.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### Group by neighbourhood postal codes

In [11]:
df = table.groupby(['Postcode','Borough'])['Neighbourhood\n'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M2H,North York,Hillcrest Village
1,M2N,North York,Willowdale South
2,M1B,Scarborough,"Rouge , Malvern"
3,M3J,North York,"Northwood Park , York University"
4,M5S,Downtown Toronto,"Harbord , University of Toronto"
5,M9B,Etobicoke,"Cloverdale , Islington , Martin Grove , Prince..."
6,M4C,East York,Woodbine Heights
7,M2M,North York,"Newtonbrook , Willowdale"
8,M8W,Etobicoke,"Alderwood , Long Branch"
9,M4M,East Toronto,Studio District


In [12]:
print(df.shape)

(103, 3)


# END OF PART 1 OF THE ASSIGNMENT

## PART 2: Geocoding the data

In [13]:
url2="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(url2)
geo_data.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [14]:
print(list(df))
print(list(geo_data))

full_table = df.set_index('Postcode').join(geo_data.set_index('Postal Code'))
full_table = full_table.sample(frac=1).reset_index(drop=True)
full_table.head(20)

['Postcode', 'Borough', 'Neighbourhood\n']
['Postal Code', 'Latitude', 'Longitude']


,Borough,Neighbourhood,Latitude,Longitude
0,North York,"Northwood Park , York University",43.767980,-79.487262
1,Downtown Toronto,"Adelaide , King , Richmond",43.650571,-79.384568
2,Central Toronto,Lawrence Park,43.728020,-79.388790
3,East Toronto,The Beaches,43.676357,-79.293031
4,North York,"Lawrence Heights , Lawrence Manor",43.718518,-79.464763
5,York,"Del Ray , Keelesdale , Mount Dennis , Silverth...",43.691116,-79.476013
6,North York,Don Mills North,43.745906,-79.352188
7,North York,"Newtonbrook , Willowdale",43.789053,-79.408493
8,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
9,Scarborough,"Rouge , Malvern",43.806686,-79.194353


# END OF PART 2 OF THE ASSIGNMENT